## Baseline model
표 형식의 데이터를 다루는 머신러닝 파이브라인의 일반적인 순서는 아래와 같다.

Data Preprocessing - Feature Engineering - Model Training - Test Model - Predict Result

## Data Preprocessing
* 결측값 -> 0으로 대체, 제품 보유여부의 정보가 없으면 보유하지 않음을 가정
* 훈련 데이터와 테스트 데이터의 병합(날짜변수로 쉽게 구분을 할 수 있음, 24개의 고객변수가 동일한 것을 그대로 병합하고 테스트에 없는 24개의 제품변수는 0으로 채움)
* 범주형 데이터는 factorize()를 통해 label encoding을 수행하고 데이터 타입이 object로 표현되는 수치형 데이터는 .unique()를 통해 특이값들을 대체하거나 제거하고 정수형 데이터로 변환한다.
* 추후, 모델 학습에 사용할 변수 이름을 features 리스트에 미리 담아둔다.

In [7]:
import pandas as pd
import numpy as np
import xgboost as xgb

np.random.seed(2018)

# 데이터를 불러옴
train = pd.read_csv('C:\\Users\\silve\\Desktop\\santander\\data\\train_ver2.csv')
test = pd.read_csv('C:\\Users\\silve\\Desktop\\santander\\data\\test_ver2.csv')

In [9]:
# 변수들을 별도로 저장해둠
prods = train.columns[24:].tolist()

# 제품 변수 결측값을 0으로 대체함
train[prods] = train[prods].fillna(0.0).astype(np.int8)

# 24개중 하나도 보유하지 않은 고객의 데이터를 제거
no_product = train[prods].sum(axis = 1) == 0
train = train[~ no_product]

# 훈련과 테스트 데이터를 통합하고, 테스트 데이터에 없는 제품의 변수는 0으로 대체하여 채운다
for col in train.columns[24:]:
    test[col] = 0
df = pd.concat([train,test], axis = 0)

# 학습에 사용할 변수를담는 list생성
features = []

# 범주형 변수를 .factorize()함수를 통해 label encoding함
categorical_cols = ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi',
                   'indext', 'conyuemp', 'canal_entrada', 'indfall', 'tipodom', 'nomprov', 'segmento']
for col in categorical_cols:
    df[col], _ = df[col].factorize(na_sentinel = -99)
features += categorical_cols

# 수치형 변수의 특이값과 결측값을 -99로 대체하고 정수형으로 변환한다
df['age'].replace(' NA', -99, inplace=True)
df['age'] = df['age'].astype(np.int8)

df['antiguedad'].replace('     NA', -99, inplace=True)
df['antiguedad'] = df['antiguedad'].astype(np.int8)

df['renta'].replace('         NA', -99, inplace=True)
df['renta'].fillna(-99, inplace=True)
df['renta'] = df['renta'].astype(float).astype(np.int8)

df['indrel_1mes'].replace('P', 5, inplace=True)
df['indrel_1mes'].fillna(-99, inplace=True)
df['indrel_1mes'] = df['indrel_1mes'].astype(float).astype(np.int8)

#학습에 사용할 수치형 변수를 features에 추가한다.
features += ['age', 'antigue', 'renta', 'ind_nuevo', 'indrel', 'indrel_1mes', 'ind_actividad_cliente']

## Feature Engineering
feature engineering에서는 모델에서 사용할 파생 변수를 생성한다.

baseline 모델에서는 24개의 고객변수와 4개의 날짜기반 파생변수 그리고 24개의 lag-1 변수를 사용한다.
